# Putting it all together - Preparing the Environment

Welcome back! In the previous sections, you've explored the foundational building blocks of developing with Llama Stack: understanding basic agents, delving into the power of ReAct agents, and seeing how the Model Context Protocol (MCP) allows seamless integration of external services as tools.

Now, it's time to bring these concepts together and prepare our environment for building a more sophisticated agent capable of interacting with multiple external services and leveraging internal knowledge. This section is dedicated to setting the stage, ensuring all the necessary components are up and running and configured correctly within Llama Stack.

By the end of this preparatory section, you will have:

* Understood the setup required for a multi-tool, RAG-enabled agent.
* Launched and verified the operation of multiple MCP servers exposing diverse functionalities.
* Configured your Llama Stack client to interact with these new tools.
* Registered the MCP servers as discoverable toolgroups within Llama Stack.
* Created and populated a Vector Database, essential for providing our agent with domain-specific knowledge via Retrieval Augmented Generation (RAG).

Think of this section as gathering and organizing all the ingredients and setting up your workspace before you start cooking a complex and exciting dish! Let's get everything ready so we can build something truly powerful in the next section.

***

### Setting up our Environment

Before we dive into the exciting part of building our multi-tool agent, let's make sure our environment is set up correctly. We'll start by importing the essential libraries and configuring our connection to the Llama Stack server, just like we did in previous labs.

In [6]:
!pip install -U llama-stack-client==0.2.7 dotenv > /dev/null 2>&1 && echo "pip Python Prerequisites installed succesfuly"
import os
from src.utils import step_printer
from termcolor import cprint
import uuid

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.event_logger import EventLogger



stream=False 
allowed_models_list=["meta-llama/Llama-3.2-3B-Instruct"]

LLAMA_STACK_SERVER='http://localhost:8321'
client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
)

selected_model = None
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
if selected_model is None:
    print("No allowed model found in the list.")


print(f"Selected model (from allowed list): {selected_model}")
            # Removed the break here to show all available models, but the selection logic remains picking the first one
SELECTED_MODEL = selected_model


pip Python Prerequisites installed succesfuly
--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
Selected model (from allowed list): meta-llama/Llama-3.2-3B-Instruct


### Launching our External Services (MCP Servers)

To create a truly capable agent, we need to connect it to the outside world! We'll be using a few specialized external services, exposed via the Model Context Protocol (MCP), to give our agent powers like checking the weather, finding locations, and accessing specific park information.

Run the following commands in your terminal to start these essential MCP servers in the background:

* `mcp-weather`: Adds real-time weather capabilities.
* `mcp-googlemaps`: Connects to Google Maps for location and direction data.
* `mcp-parks-info`: Offers RAG-based information about our fictional parks using RAG.

In [7]:
%%bash
source ~/.bashrc
podman run -d --replace --name mcp-weather --network=host quay.dev.demo.redhat.com/rhdp/mcp-weather:latest --port 8005 
podman run -d --replace --name mcp-googlemaps-sse --network=host -e GOOGLE_MAPS_API_KEY=$GOOGLE_MAPS_API_KEY -e MCP_PORT=8006 quay.dev.demo.redhat.com/rhdp/mcp-googlemaps-sse:latest
podman run -d --replace --name mcp-parks-info --network=host -e PORT=8007 quay.dev.demo.redhat.com/rhdp/mcp-parks-info:latest

podman ps

time="2025-05-17T18:11:30Z" level=warning msg="StopSignal SIGTERM failed to stop container mcp-weather in 10 seconds, resorting to SIGKILL"


5885c6b7f3ff2e1714a65e5bcbd433f54a8c7e827da75b5408f840e04175e968


Trying to pull quay.dev.demo.redhat.com/rhdp/mcp-googlemaps-sse:latest...
Getting image source signatures
Copying blob sha256:9072e2b43e428f5b955a494f416c2f7ce1c7895cabe222f3ab58f15d5936adcb
Copying blob sha256:af70d51bc91f4dfec8899951d2ec3b55a5a0ac08279db44e5a797a1028220413
Copying blob sha256:b0f6f1c319a1570f67352d490370f0aeb5c0e67a087baf2d5f301ad51ec18858
Copying blob sha256:4f361b47bbd6d447cf938eeb7945ff5851832edf4daa196acfcf963fd2377339
Copying blob sha256:60f51d565ac390f55b710d0403051fbdc8c67dcd340a3435543be93637630843
Copying blob sha256:043debd7ea09cd971f3eae8dc4a62ccc5f581617ece07246fd080b0f62d56356
Copying blob sha256:26d756338f506b2f00ec798097134f931ddefd05edfe7dd957b6acedd0539fa6
Copying blob sha256:4b889521ebfd129dd84cc5c9295deaf663ca3d611ba1f4168aae2e5365c5c6bb
Copying blob sha256:a391a831567b9a1af33465a7054129da83dea008c59b4f24d9160162c0bf8595
Copying blob sha256:f29bf23b693f430dfb25292d9e0b29e48d1d524eb1b775755f826f494802884e
Copying config sha256:8f67da8740421a982b2ee3

4d5933bac1a3ff8724736a2e75fe8a4244455ab67e70e5b53c2ab0736d6b60fa


Trying to pull quay.dev.demo.redhat.com/rhdp/mcp-parks-info:latest...
Getting image source signatures
Copying blob sha256:d2356d971238ca958d4fbdaf64c3548328e1a1c1d27663aaadd8dc8eccc78a00
Copying blob sha256:a8d0f3a3203486c1cc2932289a0b552768e6e3c52076b384c3e7731e77fe39ae
Copying blob sha256:08f144d6f626c0e3bdc90940ee48538bf87b15bfc122b93f96a5a2f1f77efd12
Copying blob sha256:3d04dbd71e156befabe4a49dbd96e04907e64132ed7043ed926d3f3724eab73a
Copying blob sha256:24626d2b50367d914dc5b504c0d949ddc1b00e25e45ac39786bbeea7a68f7045
Copying blob sha256:7f48b8fefae7246ca5e3650ea24027cfb03d5ccda5146f6aedb5b8b2e4d356fe
Copying config sha256:cf772fa3de2a10d8b9d6ff362115252d6b129004180029bc00839ea422de5cc9
Writing manifest to image destination


84285ed3fb4d3aaadcd8e25438372c58be874de27a6230fb8b2813d871b34d82
CONTAINER ID  IMAGE                                                    COMMAND               CREATED                 STATUS                 PORTS               NAMES
e5dcde41db91  docker.io/llamastack/distribution-ollama:0.2.7           --port 8321 --env...  About an hour ago       Up About an hour                           llamastack-ollama
5885c6b7f3ff  quay.dev.demo.redhat.com/rhdp/mcp-weather:latest         --port 8005           14 seconds ago          Up 15 seconds          8000/tcp, 8080/tcp  mcp-weather
4d5933bac1a3  quay.dev.demo.redhat.com/rhdp/mcp-googlemaps-sse:latest                        8 seconds ago           Up 9 seconds                               mcp-googlemaps-sse
84285ed3fb4d  quay.dev.demo.redhat.com/rhdp/mcp-parks-info:latest                            Less than a second ago  Up Less than a second  8000/tcp, 8080/tcp  mcp-parks-info


### Verify Servers are Running

It's always a good practice to quickly check if our external services have started successfully. We can do this by attempting to connect to their status endpoints. Run the code below to verify they are live and ready.

In [9]:

!echo Testing MCP-WEATHER ; curl --max-time 1 http://localhost:8005/sse 2>/dev/null
!echo Testing MCP-GOOGLEMAPS-sse ; curl --max-time 1 http://localhost:8006/sse 2>/dev/null 
!echo Testing mcp-parks-info ; curl --max-time 1 http://localhost:8007/sse 2>/dev/null 


Testing MCP-WEATHER
event: endpoint
data: /messages/?session_id=a8fc55f4985b42acb62ff909a8117fe0

Testing MCP-GOOGLEMAPS-sse
event: endpoint
data: /message?sessionId=09dce9dc-3235-4e55-a9b8-654994114e6a

Testing mcp-parks-info
event: endpoint
data: /messages/?session_id=8e44a69a68e84310bdf8d19ee24cf04a



### Connect to Llama Stack and Select Our Model

Now that our external services are running, let's establish the connection from our notebook to the Llama Stack server and select the powerful language model our agent will use for reasoning and action.

In [10]:
!pip install -U llama-stack-client==0.2.7 dotenv > /dev/null 2>&1 && echo "pip Python Prerequisites installed succesfuly"

import os
from src.utils import step_printer
from termcolor import cprint
import uuid

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.event_logger import EventLogger



stream=False 
allowed_models_list=["granite3.2:8b"]
#allowed_models_list=["meta-llama/Llama-3.2-3B-Instruct"]

LLAMA_STACK_SERVER='http://localhost:8321'
client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
)

selected_model = None
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
if selected_model is None:
    print("No allowed model found in the list.")


print(f"Selected model (from allowed list): {selected_model}")
            # Removed the break here to show all available models, but the selection logic remains picking the first one
SELECTED_MODEL = selected_model

vector_db_id = "Our_Parks_DB"
query_config = {
    "query_generator_config": {
        "type": "default",
        "separator": " "
    },
    "max_tokens_in_context": 300,
    "max_chunks": 2
}


pip Python Prerequisites installed succesfuly
--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
Selected model (from allowed list): granite3.2:8b


### Making External Services Available as Tools

This is where the magic happens! We need to tell Llama Stack about the external services we just started. By registering them as 'toolgroups', Llama Stack understands their capabilities and can make them available for our agent to discover and use in its problem-solving process.

In [11]:
client.toolgroups.register(
        toolgroup_id="mcp::mcp-weather",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8005/sse"},
    )

client.toolgroups.register(
        toolgroup_id="mcp::mcp-googlemaps",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8006/sse"},
    )
client.toolgroups.register(
        toolgroup_id="mcp::mcp-parks-info",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8007/sse"},
    )

## Building Our Agent's Knowledge Base (RAG)

A truly smart agent can also access specific, domain-level knowledge beyond its initial training. We'll achieve this using Retrieval Augmented Generation (RAG). The first step is to create and populate a Vector Database with the information our agent might need – in this case, details about our parks!

In [12]:
#Unregistering vector database in case we are running this over and over again as part of a learning experience

for vector_db_id in client.vector_dbs.list():
    print(f"Unregistering vector database: {vector_db_id.identifier}")
    client.vector_dbs.unregister(vector_db_id=vector_db_id.identifier)

# Select a VectorDB provider from availalbe providers
providers = client.providers.list()
vector_providers = []
for provider in client.providers.list():
    if provider.api == "vector_io":
        #print(f"Found VectorDB provider: {provider.provider_id}\n")  # Simple print
        vector_providers.append(provider)

# In this example, we only have one provider, but on other server we might have many. here, we simply select the first one.
selected_vector_provider = vector_providers[0]

# register our DB
vector_db_id = "Our_Parks_DB"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id=selected_vector_provider.provider_id,
)

# Injest documents from local directory (this is so you can test inserting changes easily)
from llama_stack_client.types import Document

files = [
    "Azure_Mongrove_Wilderness.md",
    "Crimson_Basin.md",
     "Granite_Spire.md",
     "Obsidian_Rainforest.md",
     "Prismatic_Painted_Prairie.md",
]

document_dirctory="assets/Parks"

# Read documents into the "documents" array
documents = [
    Document(
        document_id=f"num-{i}",
        content=open(os.path.join(document_dirctory, file), 'r', encoding='utf-8').read(),
        mime_type="text/plain",
        metadata={},
    )
    for i, file in enumerate(files)
]

# Insert the documents into the vectorDB
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=300,
)

print(f"Created {len(documents)} documents from local files into {vector_db_id}")


#print(documents)

Unregistering vector database: test_vector_db_33e67287-a7ac-4845-88e3-0c8895b6d28a
Created 5 documents from local files into Our_Parks_DB


# Summary: Putting it all together - Preparing the Environment

In this preparatory section, you've successfully laid the groundwork for building a more advanced agent within the Llama Stack framework. You've taken the essential steps to integrate multiple external services and prepare a knowledge base for your agent to utilize.

You have accomplished the following key tasks:

* **Deployed Multiple MCP Servers:** You initiated and confirmed the operation of several MCP-enabled containers (`mcp-weather`, `mcp-googlemaps`, `mcp-parks-info`), demonstrating how Llama Stack can connect to a variety of external functionalities.
* **Configured the Llama Stack Client:** You set up your Python client, specifying the server address and selecting the appropriate language model that will power your agent.
* **Registered External Tools:** You registered the different MCP servers as toolgroups within Llama Stack, making the functions they expose available for discovery and use by agents.
* **Established a Knowledge Base:** You created and populated a Vector Database with relevant documents, setting up the critical component for Retrieval Augmented Generation (RAG) to give your agent access to specific information.

By completing these steps, your environment is now ready for you to build and experiment with an agent that can combine reasoning, acting with diverse tools (both external services via MCP and internal RAG), and leveraging a custom knowledge base. This is a significant step towards creating more intelligent and capable AI applications with Llama Stack!

### Cleaning Up (Optional)

If you need to stop and remove the MCP server containers later, you can use these commands. This is helpful for resetting your environment.

In [ ]:
DON"T RUN THIS IF YOU DON"T MEAN TO, IT WILL DELETE YOUR DATABASES

toolgroups_to_unregister = [
    "mcp::mcp-weather",
    "mcp::mcp-googlemaps",
    "mcp::mcp-parks-info",
]

for toolgroup_id in toolgroups_to_unregister:
    try:
        print(f"Attempting to unregister toolgroup: {toolgroup_id}")
        client.toolgroups.unregister(toolgroup_id=toolgroup_id)
        print(f"Successfully unregistered toolgroup: {toolgroup_id}")
    except Exception as e:
        print(f"Failed to unregister toolgroup {toolgroup_id}. Error: {e}")

print("\nFinished attempting to unregister toolgroups.")

In [ ]:
THIS WILL DELETE ALL OF YOUR DATABASES ONLY DO THIS IF YOU MEAN TOOOOO 
#Unregister all vector databases (THIS IS FOR DEBUG NOT FOR LAB)
for vector_db_id in client.vector_dbs.list():
    print(f"Unregistering vector database: {vector_db_id.identifier}")
    client.vector_dbs.unregister(vector_db_id=vector_db_id.identifier)